In [1]:
#import schedule, datetime, time
import  datetime, time, smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import requests, json,csv
import pandas as pd 
from botocore.client import Config
import ibm_boto3
import ibm_db
#from urllib.request import urlopen

yesterday = datetime.date.today() - datetime.timedelta(1)
outyear = str(yesterday.year)
outmonth = str(yesterday.month)
outday = str(yesterday.day)
outdate =  outyear + outmonth.rjust(2, '0') + outday.rjust(2, '0')    

###############################################################################
def APIcall(clientId, predictionLevel, threshold, email): # function for pulling data   
    url = 'https://api.weather.com/v3/forecast/outage/daily/' + str(clientId) + '?predictionLevel=' + str(predictionLevel)
    url = url + '&startDateTime=' + time.strftime("%Y%m%d%H%M") + '&format=json&apiKey=YOUR_WEATHER_API_KEY_HERE'         
    print ("Pulling Outage Prediction Forecast data..." + time.strftime("%b %d,%Y %H:%M") + " for: " + clientId + " " + predictionLevel + " " + threshold + " " + email)
    print (url)
    data = requests.get(url).json() #Our API Call
    clientList = data['predictions']
    outageDetected = 0
    for x in range(0, len(clientList)):
        forecastDateAndTime = data['metadata']['modelDateUtc']
        forecastDay = data['predictions'][x]['forecastDay']
        totalOutageCount = str(data['predictions'][x]['totalOutageCount'])
        mobilizationLevel = data['predictions'][x]['mobilizationLevel']
        outageRegionId = data['predictions'][x]['outageRegionId']
        #print(totalOutageCount)        
        if(int(totalOutageCount) > int(threshold)):            
            outageDetected = 1
            print('##########################   ALERT!!!  Over threshold of: ' + str(threshold) + ' for day ' + str(x) + '################################')
            #Email update
            emails = [email]            
            for email in emails:
                projectedDay = str(x + 1)
                if(projectedDay == '1'):
                    projectedDay = '24 hours'
                else: 
                    if(projectedDay == '2'):
                        projectedDay = '2 days (48 hours from now)'
                    else:
                        projectedDay = '3 days (72 hours from now)'
                twentyFourHour = data['predictions'][0]['totalOutageCount']
                fortyEightHour = data['predictions'][1]['totalOutageCount']
                seventyTwoHour = data['predictions'][2]['totalOutageCount']
                seventyTwoHourHourTotal = twentyFourHour + fortyEightHour + seventyTwoHour
                sub = str(clientId) + ' Outage Alert: 24 hours: ' + str(twentyFourHour) + ' | 48 Hours: ' + str(fortyEightHour) + ' | 72 hours: ' + str(seventyTwoHour) 
                bod = 'Client: ' + clientId + '\nOutage Threshold Amount: ' + threshold + '\n72 Hour cumulative: ' + str(seventyTwoHourHourTotal)              
                bod = bod + '\n\nAlert Timestamp: ' + time.strftime("%b %d %Y- %H:%M:%S") + ' UTC'
                bod = bod + '\nForecast Timestamp: ' + forecastDateAndTime
                bod = bod + '\n\nTime Period Predictions'
                bod = bod + '\n1 - 24: ' + str(twentyFourHour)
                bod = bod + '\n24 to 48: '+ str(fortyEightHour)
                bod = bod + '\n48 to 72: '+ str(seventyTwoHour)
                
                #Next we call for substation detail
                url2 = 'https://api.weather.com/v3/forecast/outage/daily/' + str(clientId) + '?predictionLevel=substation'
                url2 = url2 + '&startDateTime=' + time.strftime("%Y%m%d%H%M") + '&format=json&apiKey=YOUR_WEATHER _KEY_HERE'         
                print ("Pulling substation data for " + str(clientId)  + ' Date Time of Request: ' + time.strftime("%b %d,%Y %H:%M"))
                print (url2)
                data2 = requests.get(url2).json() #Our API Call
                subStations = data2['predictions']
                bod = bod + '\n\nSub Station Details'
                convertedDay = ''
                for y in range(0, len(subStations)):
                    subStationForecastDay = data2['predictions'][y]['forecastDay']
                    if(subStationForecastDay == 1): 
                        convertedDay = '1 to 24'
                    else:
                        if(subStationForecastDay == 2):
                            convertedDay = '24 to 48'
                        else:
                            convertedDay = '48 to 72 hours'                            
                    subStationMobilizationLevel = data2['predictions'][y]['mobilizationLevel']
                    subStationTotalOutageCount = data2['predictions'][y]['totalOutageCount']
                    subStationOutageRegionId = data2['predictions'][y]['outageRegionId']
                    subStationOutageRegionName = data2['predictions'][y]['outageRegionName']                    
                    bod = bod + '\nForecast Timeframe: ' + convertedDay + ' SubStation: ' + subStationOutageRegionName
                    bod = bod + ' Total Predicted Outages: ' + str(subStationTotalOutageCount)
                send_email('YOUR_GMAIL_USERNAME_HERE', 'YOUR_GMAIL_PASSWORD_HERE', email, sub, bod)    
def send_email(user, pwd, recipient, subject, body):
    FROM = user
    TO = recipient if isinstance(recipient, list) else [recipient]
    SUBJECT = subject
    TEXT = body

    # Prepare actual message
    message = """From: %s\nTo: %s\nSubject: %s\n\n%s
    """ % (FROM, ", ".join(TO), SUBJECT, TEXT)
    
    server = smtplib.SMTP("smtp.gmail.com", 587)
    server.ehlo()
    server.starttls()
    server.login(user, pwd)
    server.sendmail(FROM, TO, message)
    server.close()
    print ('successfully sent the mail to: ' + recipient)
    
# This is the section where we import the Data file called notificationList.csv from Object Storage. You choose Insert Pandas Data Frame From imported file.
###############################################################################
conn = ibm_db.connect("DATABASE=YOUR_DB2_DB_NAME_HERE;HOSTNAME=YOUR_DB2_HOSTNAME_HERE;PORT=YOUR_DB2_POT_HERE;PROTOCOL=TCPIP;UID=YOUR_DB2_DB_USERNAME_HERE;PWD=YOUR_DB2_PASSWORD_HERE", "YOUR_DB2_DB_USERNAME_HERE", "YOUR_DB2_PASSWORD_HERE")
results = []
if conn:
    sql = "SELECT * FROM ALERT_LIST_OUTAGE_PREDICTION"
    stmt = ibm_db.exec_immediate(conn, sql)
    result = ibm_db.fetch_both(stmt)
    while( result ):
        clientId = result[0]
        predictionLevel = result[1]
        threshold = result[2]
        email = result[3]
        print ("Result from Query:",clientId , predictionLevel, threshold, email)
        results.append(result)
        result = ibm_db.fetch_both(stmt)
        
        
#Iterate through the list created from the database results.
for a in results:
    APIcall(str(a[0]), str(a[1]), str(a[2]), str(a[3]))



Exception: [IBM][CLI Driver] SQL1337N  The service "YOUR_DB2_POT_H" was not found. SQLCODE=-1337